# Comparing spatial pattern of velocity response to forcing

In this notebook, we'll use the `iceutils` package (Bryan Riel) to invert continuous time-varying surface velocity fields on Helheim Glacier.  We'll then process several observational datasets (gathered by Denis Felikson) using the `nifl` module (Lizz Ultee) and compare time series of these variables against surface velocity at several points.  Finally, we'll visualize spatial differences in the relationship between surface velocity and each hypothesised forcing variable.

Last updated: 6 Jan 2022 by Lizz Ultee

### Import the necessary packages

In [ ]:
from netCDF4 import Dataset
from scipy import interpolate, ndimage
import pyproj as pyproj
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
from matplotlib.colors import LightSource
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import iceutils as ice
import nifl_helper as nifl

### Define where the necessary data lives

In [ ]:
flowline_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Felikson-flowlines/netcdfs/glaciera199.nc'
velocity_fpath='/Volumes/Backup Plus/Large data moved 20220331/Gld-Stack/'
gl_bed_fpath ='/Users/lizz/Documents/GitHub/Data_unsynced/BedMachine-Greenland/BedMachineGreenland-2017-09-20.nc'
catchment_smb_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Helheim-processed/smb_rec._.BN_RACMO2.3p2_ERA5_3h_FGRN055.1km.MM.csv'
runoff_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Helheim-processed/runoff._.BN_RACMO2.3p2_ERA5_3h_FGRN055.1km.MM.csv'
termini_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Helheim-processed/HLM_terminus_widthAVE.csv'

### Define the domain of analysis

We will analyse along flowlines defined by Denis Felikson in his previous work, saved and shared as NetCDF files.  The flowlines are numbered 01-10 across the terminus; flowline 05 is close to the middle.  Note that Helheim Glacier has two large branches.  For now we'll study the main trunk, `glaciera199.nc`.  The more southerly trunk is `glacierb199.nc`.

In [ ]:
ncfile = Dataset(flowline_fpath, 'r')
xh = ncfile['flowline05'].variables['x'][:]
yh = ncfile['flowline05'].variables['y'][:]
ncfile.close()

In [ ]:
## Define points at which to extract
upstream_max = 500 # index of last xh,yh within given distance of terminus--pts roughly 50m apart
xys = [(xh[i], yh[i]) for i in range(0, upstream_max, 20)][2::]

## Import and invert velocity observations

In [ ]:
## Set up combined hdf5 stack
hel_stack = ice.MagStack(files=[velocity_fpath+'vx.h5', velocity_fpath+'vy.h5'])
data_key = 'igram' # B. Riel convention for access to datasets in hdf5 stack

In [ ]:
# Create an evenly spaced time array for time series predictions
t_grid = np.linspace(hel_stack.tdec[0], hel_stack.tdec[-1], 1000)

# First convert the time vectors to a list of datetime
dates = ice.tdec2datestr(hel_stack.tdec, returndate=True)
dates_grid = ice.tdec2datestr(t_grid, returndate=True)

# Build the collection
collection = nifl.build_collection(dates)

# Construct a priori covariance
Cm = nifl.computeCm(collection)
iCm = np.linalg.inv(Cm)

# Instantiate a model for inversion
model = ice.tseries.Model(dates, collection=collection)

# Instantiate a model for prediction
model_pred = ice.tseries.Model(dates_grid, collection=collection)

## Access the design matrix for plotting
G = model.G

# Create lasso regression solver that does the following:
# i) Uses an a priori covariance matrix for damping out the B-splines
# ii) Uses sparsity-enforcing regularization (lasso) on the integrated B-splines
solver = ice.tseries.select_solver('lasso', reg_indices=model.itransient, penalty=0.05,
                                   rw_iter=1, regMat=iCm)

Now that we are set up with our data and machinery, we'll ask the inversion to make us a continuous time series of velocity at each point we wish to study.

In [ ]:
preds = []
for j, xy in enumerate(xys):
    try:
        pred, st, lt = nifl.VSeriesAtPoint(xy, vel_stack=hel_stack, collection=collection, 
                                  model=model, model_pred=model_pred, solver=solver, 
                                  t_grid=t_grid, sigma=2.5, data_key='igram')
        preds.append(pred)
    except AssertionError: # catches failed inversion
        print('Insufficient data for point {}. Removing'.format(j))
        xys.remove(xy)
        continue

Test to confirm that `xys` has been trimmed to only those points for which a valid prediction can be generated -- the below should return `True`.

In [ ]:
len(xys)==len(preds)

## Comparison data sets

### Bed topography

Mostly we will use this for plotting and for defining a standard coordinate system.  However, future analyses could combine bed topography with calving position or other variables to analyse effect on surface velocity.

In [ ]:
## Read in and interpolate BedMachine topography
fh = Dataset(gl_bed_fpath, mode='r')
xx = fh.variables['x'][:].copy() #x-coord (polar stereo (70, 45))
yy = fh.variables['y'][:].copy() #y-coord
s_raw = fh.variables['surface'][:].copy() #surface elevation
h_raw=fh.variables['thickness'][:].copy() # Gridded thickness
b_raw = fh.variables['bed'][:].copy() # bed topo
thick_mask = fh.variables['mask'][:].copy()
ss = np.ma.masked_where(thick_mask !=2, s_raw)#mask values: 0=ocean, 1=ice-free land, 2=grounded ice, 3=floating ice, 4=non-Greenland land
hh = np.ma.masked_where(thick_mask !=2, h_raw) 
bb = b_raw #don't mask, to allow bed sampling from modern bathymetry (was subglacial in ~2006)
fh.close()

In [ ]:
## Interpolate in area of Helheim
xl, xr = 6100, 6600
yt, yb = 12700, 13100
x_hel = xx[xl:xr]
y_hel = yy[yt:yb]
s_hel = ss[yt:yb, xl:xr]
b_hel = bb[yt:yb, xl:xr]
S_helheim = interpolate.RectBivariateSpline(x_hel, y_hel[::-1], s_hel.T[::,::-1]) #interpolating surface elevation provided
B_helheim = interpolate.RectBivariateSpline(x_hel, y_hel[::-1], b_hel.T[::,::-1]) #interpolating bed elevation provided

### Catchment-integrated SMB

We load in a 1D timeseries of surface mass balance integrated over the whole Helheim catchment.  This data is monthly surface mass balance from the HIRHAM5 model, integrated over the Helheim catchment defined by K. Mankoff, with processing steps (coordinate reprojection, Delaunay triangulation, nearest-neighbor search and area summing) in `catchment-integrate-smb.py`.

In [ ]:
smb_racmo = pd.read_csv(catchment_smb_fpath, index_col=0, parse_dates=True)
smb_tr = smb_racmo.loc[smb_racmo.index.year >= 2006]
smb = smb_tr.loc[smb_tr.index.year <2018].squeeze() # trim dates to overlapping period

smb_d = [d.utctimetuple() for d in smb.index]
smb_d_interp = [ice.timeutils.datestr2tdec(d[0], d[1], d[2]) for d in smb_d]
smb_func = interpolate.interp1d(smb_d_interp, smb)

The time series we analyse here are autocorrelated. We must compute a correction factor to the significance limits based on the lag-1 autocorrelation, as described in Dean & Dunsmuir (2016).

In [ ]:
a_vel = sm.tsa.stattools.acf(np.diff(preds[0]['full']))[1]
b_smb = sm.tsa.stattools.acf(np.diff(smb))[1]
F_smb = np.sqrt((1+(a_vel*b_smb))/(1-(a_vel*b_smb)))

Now, we compute the normalized cross-correlation between catchment-integrated SMB and surface velocity at each point along the flowline.  We will draw on the inverted velocity series saved in `preds` above.  We save the value of the maximum normalized cross-correlation, and the value in days of the lag where it occurs, to compare with other variables later.  We also test whether each saved value is significantly different from 0, with the confidence interval around 0 modified as above.

In [ ]:
smb_corr_amax = []
smb_lag_amax = []
smb_significance = []

for xy, pred in zip(xys, preds):
    corr, lags, ci = nifl.Xcorr1D(xy, series_func=smb_func, series_dates=smb_d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=1, normalize=True, pos_only=True)
    ci_mod = F_smb*np.asarray(ci)
    smb_corr_amax.append(corr[abs(corr).argmax()])
    smb_lag_amax.append(lags[abs(corr).argmax()])
    smb_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

### Runoff

We import monthly runoff from the RACMO model, integrated over the Helheim catchment and shared as a CSV by Denis Felikson.  Because this data is catchment-integrated, we interpolate a single 1D time series that will be used at all points.

In [ ]:
runoff_racmo = pd.read_csv(runoff_fpath, index_col=0, parse_dates=True)
runoff_tr = runoff_racmo.loc[runoff_racmo.index.year >= 2006]
runoff = runoff_tr.loc[runoff_tr.index.year <2018].squeeze()

runoff_d = [d.utctimetuple() for d in runoff.index]
d_interp = [ice.timeutils.datestr2tdec(d[0], d[1], d[2]) for d in runoff_d]
runoff_func = interpolate.interp1d(d_interp, runoff)

Again, we compute a correction factor to account for autocorrelated data in the 95% confidence interval around 0.

In [ ]:
b_runoff = sm.tsa.stattools.acf(np.diff(runoff))[1]
F_runoff = np.sqrt((1+(a_vel*b_runoff))/(1-(a_vel*b_runoff)))

We compute the normalized cross-correlation between catchment-integrated runoff and surface velocity at each same point.  Again we save the value of the maximum normalized cross-correlation, and the value in days of the lag where it occurs, to compare with other variables.

In [ ]:
runoff_corr_amax = []
runoff_lag_amax = []
runoff_significance = []

for xy, pred in zip(xys, preds):
    corr, lags, ci = nifl.Xcorr1D(xy, series_func=runoff_func, series_dates=d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=1, normalize=True, pos_only=True)
    ci_mod = F_runoff*np.asarray(ci)
    runoff_corr_amax.append(corr[abs(corr).argmax()])
    runoff_lag_amax.append(lags[abs(corr).argmax()])
    runoff_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

### Terminus position change

We import width-averaged terminus position change processed by Leigh Stearns.  These data give terminus position in km from a baseline, so they do not need to be processed into a coordinate system.

In [ ]:
termini = pd.read_csv(termini_fpath, index_col=0, parse_dates=True, usecols=[0,1])
trmn = termini.loc[termini.index.year >= 2006]
tm = trmn.loc[trmn.index.year <2017].squeeze()

## smooth a little to make more comparable with SMB and runoff
td = tm.rolling('10D').mean() # approximately 3 measurements per window

termini_d = [d.utctimetuple() for d in td.index]
tm_d_interp = [ice.timeutils.datestr2tdec(d[0], d[1], d[2]) for d in termini_d]
termini_func = interpolate.interp1d(tm_d_interp, td)

And the confidence interval correction factor for these data:

In [ ]:
b_terminus = sm.tsa.stattools.acf(np.diff(td))[1]
F_terminus = np.sqrt((1+(a_vel*b_terminus))/(1-(a_vel*b_terminus)))

Finally, finding the maximum cross-correlation and the (positive) lag at which it occurs, as for other variables.

In [ ]:
terminus_corr_amax = []
terminus_lag_amax = []
terminus_significance = []

for xy, pred in zip(xys, preds):
    corr, lags, ci = nifl.Xcorr1D(xy, series_func=termini_func, series_dates=tm_d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=1, normalize=True, pos_only=True)
    ci_mod = F_terminus *np.asarray(ci)
    terminus_corr_amax.append(corr[abs(corr).argmax()])
    terminus_lag_amax.append(lags[abs(corr).argmax()])
    terminus_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

## Spatial pattern of cross-correlation (Figure 2)

Let's compare the patterns of cross-correlation for each variable, marking whether each value is significant.  Here we use a filled dot to indicate values significantly different from 0, and a cross to indicate values not significantly different from 0.  We will produce Figure 2 of the Ultee et al manuscript.

In [ ]:
div_colors = 'RdBu' # choose divergent colormap for xcorr
corrnorm_min, corrnorm_max = -0.3, 0.3
lag_colors = 'Greens'
lagnorm_min, lagnorm_max = 0, 365

sig_markers = ['o', 'x']

ls = LightSource(azdeg=225, altdeg=80)

In [ ]:
## set matplotlib font size defaults
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
## black-white hillshade topo underneath
rgb2 = ls.shade(np.asarray(b_hel), cmap=plt.get_cmap('gray'), blend_mode='overlay',
               dx=np.mean(np.diff(x_hel)), dy=np.mean(np.diff(y_hel)), vert_exag=5.)

fig, ((ax1, ax2, ax3), (ax4,ax5,ax6)) = plt.subplots(nrows=2,ncols=3, figsize=(12, 8), 
                                                     # constrained_layout=True, 
                                                     sharex=True, sharey=True,
                                                     gridspec_kw={'wspace':0.01})
    
ax1.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc1 = ax1.scatter(np.asarray(xys)[smb_significance,0], np.asarray(xys)[smb_significance,1], 
                  c=np.asarray(smb_corr_amax)[smb_significance], cmap=div_colors, marker=sig_markers[0], 
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax1.scatter(np.asarray(xys)[np.invert(smb_significance),0], np.asarray(xys)[np.invert(smb_significance),1], 
                  c=np.asarray(smb_corr_amax)[np.invert(smb_significance)], cmap=div_colors, marker=sig_markers[1], 
                  vmin=corrnorm_min, vmax=corrnorm_max) #different marker for insig values
ax1.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
        ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
       ylabel='Northing [km]', title='Catchment SMB')

ax2.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc2 = ax2.scatter(np.asarray(xys)[runoff_significance,0], np.asarray(xys)[runoff_significance,1], 
                  c=np.asarray(runoff_corr_amax)[runoff_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax2.scatter(np.asarray(xys)[np.invert(runoff_significance),0], np.asarray(xys)[np.invert(runoff_significance),1],
            c=np.asarray(runoff_corr_amax)[np.invert(runoff_significance)], cmap=div_colors, marker=sig_markers[1],
            vmin=corrnorm_min, vmax=corrnorm_max) # distinguish insig values
ax2.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
       title='Catchment runoff')

ax3.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc3 = ax3.scatter(np.asarray(xys)[terminus_significance,0], np.asarray(xys)[terminus_significance,1], 
                  c=np.asarray(terminus_corr_amax)[terminus_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax3.scatter(np.asarray(xys)[np.invert(terminus_significance),0], np.asarray(xys)[np.invert(terminus_significance),1],
            c=np.asarray(terminus_corr_amax)[np.invert(terminus_significance)], cmap=div_colors, marker=sig_markers[1],
            vmin=corrnorm_min, vmax=corrnorm_max)

div3 = make_axes_locatable(ax3)
cax3 = div3.append_axes("right", size="5%", pad=0.1)
cb3 = fig.colorbar(sc3, cax=cax3)
cb3.ax.set_ylabel('AMax. xcorr')
ax3.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
       title='Terminus position', aspect=1.)

## SECOND ROW
ax4.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc4 = ax4.scatter(np.asarray(xys)[smb_significance,0], np.asarray(xys)[smb_significance,1], 
                  c=np.asarray(smb_lag_amax)[smb_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax4.scatter(np.asarray(xys)[np.invert(smb_significance),0], np.asarray(xys)[np.invert(smb_significance),1], 
            c=np.asarray(smb_lag_amax)[np.invert(smb_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax4.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]', ylabel='Northing [km]')

ax5.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc5 = ax5.scatter(np.asarray(xys)[runoff_significance,0], np.asarray(xys)[runoff_significance,1], 
                  c=np.asarray(runoff_lag_amax)[runoff_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax5.scatter(np.asarray(xys)[np.invert(runoff_significance),0], np.asarray(xys)[np.invert(runoff_significance),1], 
            c=np.asarray(runoff_lag_amax)[np.invert(runoff_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax5.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]')

ax6.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc6 = ax6.scatter(np.asarray(xys)[terminus_significance,0], np.asarray(xys)[terminus_significance,1], 
                  c=np.asarray(terminus_lag_amax)[terminus_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax6.scatter(np.asarray(xys)[np.invert(terminus_significance),0], np.asarray(xys)[np.invert(terminus_significance),1], 
            c=np.asarray(terminus_lag_amax)[np.invert(terminus_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)

div6 = make_axes_locatable(ax6)
cax6 = div6.append_axes("right", size="5%", pad=0.1)
cb6 = fig.colorbar(sc6, cax=cax6)
cb6.ax.set_ylabel('Lag [d] at peak xcorr')
cb6.set_ticks([0, 60, 120, 180, 240, 300, 360])
ax6.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]', aspect=1.)
plt.tight_layout()
# plt.show()
# plt.savefig('/Users/lizz/Desktop/20210204-helheim-xcorr_lag_composite')

## Annual chunks to compare changing seasonal cycle

We break signals into annual subsets and compute the cross-correlation signal for each single year of data.  We will analyse patterns of these correlations at each point along flow, with color in the plot (Figure 3) indicating position.

TODO: show map plot earlier so that lines can be matched with their associated point.

In [ ]:
annual_vals = {i: {'smb': [], 'runoff': [], 'terminus': []} 
               for i in range(len(xys))}
date_chks = range(2009, 2018)

for j in range(len(xys)):
    point_to_plot = j
    v_local = preds[point_to_plot]
    a_vel = sm.tsa.stattools.acf(np.diff(v_local['full']))[1]

    F_smb = np.sqrt((1+(a_vel*b_smb))/(1-(a_vel*b_smb))) ## compute CI correction factor using local velocity series
    F_runoff = np.sqrt((1+(a_vel*b_runoff))/(1-(a_vel*b_runoff)))
    F_terminus = np.sqrt((1+(a_vel*b_terminus))/(1-(a_vel*b_terminus)))
    
    
    smb_annual_corrs = []
    smb_annual_lags = []
    smb_annual_ci = []
    for i in range(len(date_chks)-1):
        corr, lags, ci = nifl.Xcorr1D(xys[point_to_plot], series_func=smb_func, series_dates=smb_d_interp, 
                                  velocity_pred=preds[point_to_plot], t_grid=t_grid, t_limits=(date_chks[i], date_chks[i+1]),
                                      diff=1, normalize=True, pos_only=True)
        ci_mod = F_smb*np.asarray(ci) # correct the significance interval
        smb_annual_corrs.append(corr)
        smb_annual_lags.append(lags)
        smb_annual_ci.append(ci_mod)
    annual_vals[j]['smb'] = smb_annual_corrs

    
    rf_annual_corrs = []
    rf_annual_lags = []
    rf_annual_ci = []
    for i in range(len(date_chks)-1):
        corr, lags, ci = nifl.Xcorr1D(xys[point_to_plot], series_func=runoff_func, series_dates=d_interp, 
                                  velocity_pred=preds[point_to_plot], t_grid=t_grid, t_limits=(date_chks[i], date_chks[i+1]),
                                      diff=1, normalize=True, pos_only=True)
        ci_mod = F_runoff*np.asarray(ci) # correct the significance interval
        rf_annual_corrs.append(corr)
        rf_annual_lags.append(lags)
        rf_annual_ci.append(ci_mod)
    annual_vals[j]['runoff'] = rf_annual_corrs
    
    
    tm_annual_corrs = []
    tm_annual_lags = []
    tm_annual_ci = []
    for i in range(len(date_chks)-1):
        corr, lags, ci = nifl.Xcorr1D(xys[point_to_plot], series_func=termini_func, series_dates=tm_d_interp, 
                                  velocity_pred=preds[point_to_plot], t_grid=t_grid, t_limits=(date_chks[i], date_chks[i+1]),
                                      diff=1, normalize=True, pos_only=True)
        ci_mod = F_terminus *np.asarray(ci) # correct the significance interval
        tm_annual_corrs.append(corr)
        tm_annual_lags.append(lags)
        tm_annual_ci.append(ci_mod)
    annual_vals[j]['terminus'] = tm_annual_corrs

Now we plot the cross-correlations of annual subsetted signals in a stack.  Each panel of the below will represent cross-correlation between velocity and a single variable (columns) for a single year (rows).  The colored lines in each panel correspond to points along the flowline defined above.

In [ ]:
clrs = plt.get_cmap('plasma')(np.array(range(len(xys)))/len(xys))
corr_line_width=2.0

fig, axs = plt.subplots(len(rf_annual_corrs), ncols=3, figsize=(8, 13), sharex=True, sharey=True)
for i in range(len(smb_annual_ci)):
    ax = axs[i][0]
    ax.axvline(x=0, color='k', alpha=0.5)
    ax.axhline(y=0, color='k', alpha=0.5)
    ax.plot(smb_annual_lags[i], smb_annual_ci[i], ls=':', color='k')
    ax.plot(smb_annual_lags[i], -1*np.array(smb_annual_ci[i]), ls=':', color='k')
    for n in range(len(xys)):
        corr_color = clrs[n]
        ax.plot(smb_annual_lags[i], annual_vals[n]['smb'][i], color=corr_color, 
                lw=corr_line_width, alpha=0.3)
#         ax.fill_between(smb_annual_lags[i], y1=annual_vals[n]['smb'][i], y2=0, 
#                         where=abs(annual_vals[n]['smb'][i])>smb_annual_ci[i], color=corr_color, alpha=0.2)
        ax.fill_between(smb_annual_lags[i], y1=-1, y2=1, 
                        where=abs(annual_vals[n]['smb'][i])>smb_annual_ci[i], color=corr_color, alpha=0.1)
    if i==0:
        ax.set(title='SMB', ylabel='xcorr')
    elif i==len(axs)-1:
        ax.set(xlabel='Lag [d]', ylabel='xcorr')
    else:
        ax.set(ylabel='xcorr')
for j in range(len(rf_annual_ci)):
    ax = axs[j][1]
    ax.axvline(x=0, color='k', alpha=0.5)
    ax.axhline(y=0, color='k', alpha=0.5)
    ax.plot(rf_annual_lags[j], rf_annual_ci[j], ls=':', color='k')
    ax.plot(rf_annual_lags[j], -1*np.array(rf_annual_ci[j]), ls=':', color='k')
    for n in range(len(xys)):
        corr_color = clrs[n]
        ax.plot(rf_annual_lags[j], annual_vals[n]['runoff'][j], color=corr_color, 
                lw=corr_line_width, alpha=0.3)
#         ax.fill_between(rf_annual_lags[j], y1=annual_vals[n]['runoff'][j], y2=0, 
#                         where=abs(annual_vals[n]['runoff'][j])>rf_annual_ci[j], color=corr_color, alpha=0.1)
        ax.fill_between(rf_annual_lags[j], y1=-1, y2=1, 
                        where=abs(annual_vals[n]['runoff'][j])>rf_annual_ci[j], color=corr_color, alpha=0.1)
    if j==0:
        ax.set(title='Runoff')
    elif j==len(axs)-1:
        ax.set(xlabel='Lag [d]')
    else:
        continue
for k in range(len(tm_annual_ci)):
    ax = axs[k][2]
    ax.axvline(x=0, color='k', alpha=0.5)
    ax.axhline(y=0, color='k', alpha=0.5)
    ax.plot(tm_annual_lags[k], tm_annual_ci[k], ls=':', color='k')
    ax.plot(tm_annual_lags[k], -1*np.array(tm_annual_ci[k]), ls=':', color='k')
    for n in range(len(xys)):
        corr_color = clrs[n]
        ax.plot(tm_annual_lags[k], annual_vals[n]['terminus'][k], color=corr_color, 
                lw=corr_line_width, alpha=0.3)
#         ax.fill_between(tm_annual_lags[k], y1=annual_vals[n]['terminus'][k], y2=0, 
#                         where=abs(annual_vals[n]['terminus'][k])>tm_annual_ci[k], color=corr_color, alpha=0.2)
        ax.fill_between(tm_annual_lags[k], y1=-1, y2=1, 
                        where=abs(annual_vals[n]['terminus'][k])>tm_annual_ci[k], color=corr_color, alpha=0.1)
    ax.text(250, 0.55, str(date_chks[k]), ha='center', size=10, weight=500, color='k')
    if k==0:
        ax.set(title='Terminus pos.')
    elif k==len(axs)-1:
        ax.set(xlabel='Lag [d]')
    else:
        continue
for ax in axs.ravel():
    ax.set(ylim=(-1,1), 
            xlim=(0,360),
            xticks=(0, 90, 180, 270, 360)
#            xticks=(-360, -180, 0, 180, 360)
           )
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(30))
plt.tight_layout()
# plt.savefig('/Users/lizz/Desktop/{}-annual_chunk-allvars.png'.format(datetime.date.today().strftime('%Y%m%d')))

## Isolate multi-annual components of signal 

The preceding analyses studied signals whose predominant variability was at seasonal scale.  Now, we will look at cross-correlations between the components of the signals that vary over longer (multi-annual) periods.  This can help us understand whether the dominant forcings of velocity variability are consistent or different across temporal scales.

First, we define a new version of our 1-D cross-correlation function.  Instead of using the `full` version of the continuous velocity series generated by `iceutils`, this one selects the `secular` and `transient` variation components (i.e. no seasonal variation).

In [ ]:
def Xcorr1D_lt(pt, series_func, series_dates, velocity_pred, t_grid, t_limits, 
               diff=1, normalize=True, pos_only=False):
    """
    Compute cross-correlation on coincident series of a 1D time series
    (e.g. catchment-integrated runoff or SMB) versus velocity at a point.

    Parameters
    ----------
    pt : tuple
        Position (x,y) at which to pull velocity series.
    series_func : interpolate.interp1d
        1D-interpolated function with values of data series over time.
    series_dates : list
        Decimal dates of data points
    velocity_series : dict
        Output of iceutils prediction.
    t_grid : ndarray
        Evenly spaced decimal times at which spline-fit velocity is sampled
    t_limits : tuple
        Start and end dates (decimal) of the time period to study
    diff : int, optional
        Number of discrete differences to apply to data. Default is 1.
        Setting diff=0 will process the input data as-is.
    normalize : bool, optional
        Whether to normalize for a cross-correlation in [-1,1]. Default is True.
        This makes the output inter-comparable with normalized output for other
        variables.  If set to False, the signal amplitude will be larger but
        the correlation values may exceed 1.
    pos_only : bool, optional
    	Whether to analyse only xcorrs with positive lag values.  Default is False.
    	This allows a bidirectional causal relationship.  For a causal relationship 
    	hypothesised to be single-directional, choose True to display only positive
    	lag values.

    Returns
    -------
    corr : array
        Cross-correlation coefficients between SMB, velocity
    lags : array
        Time lag for each correlation value
    ci : array
        Confidence intervals for evaluation

    """
    t_min = max(min(series_dates), t_limits[0])
    t_max = min(max(series_dates), t_limits[1])
    coincident_dates = np.asarray([t for t in t_grid if (t>=t_min and t<t_max)])
    coincident_series = series_func(coincident_dates) # sample at same dates as velocity series
    
    vel_longterm = velocity_pred['secular'] + velocity_pred['transient']
    series_diff = np.diff(coincident_series, n=diff)
    vel_series_0 = vel_longterm[np.where(t_grid>=t_min)]
    vel_series = vel_series_0[np.where(t_grid[np.where(t_grid>=t_min)]<t_max)] # trim dates to match t_limits
    vel_diff = np.diff(vel_series, n=diff)
    if normalize:
        series_diff = (series_diff-np.mean(series_diff)) / (np.std(series_diff)*len(series_diff))
        vel_diff = (vel_diff-np.mean(vel_diff)) / (np.std(vel_diff))
    corr = np.correlate(series_diff, vel_diff, mode='full')
    lags = range(int(-0.5*len(corr)), int(0.5*len(corr)+1))
    ci = [2/np.sqrt(len(coincident_series)-abs(k)) for k in lags]

    ## convert lags to physical units
    lags = np.mean(np.diff(t_grid))*365.26*np.asarray(lags)
    
    if pos_only:
    	corr = corr[np.argwhere(lags>=0)].squeeze()
    	ci = np.asarray(ci)[np.argwhere(lags>=0)].squeeze()
    	lags = lags[np.argwhere(lags>=0)].squeeze()
    
    return corr, lags, ci

So, the velocity signal will already be using only the long-term-varying components.  Our time series of system variables have not been generated with `iceutils`, so instead of using component selection on them, we will apply a simple boxcar filter to remove short-term variation.

In [ ]:
t_grid_trimmed = t_grid[np.argwhere(t_grid<2017)] ## valid range for interpolated funcs
window = int(1./np.mean(np.diff(t_grid_trimmed.squeeze()))) # set window size to the number of time steps in 

In [ ]:
## Low-frequency SMB variability
smb_evensampled = 1E-9*np.array(smb_func(t_grid_trimmed).squeeze())
smb_filtered = ndimage.uniform_filter1d(smb_evensampled, size=window)
smb_lowfreq = interpolate.UnivariateSpline(t_grid_trimmed, smb_filtered, s=0)

a_vel_lt = sm.tsa.stattools.acf(preds[0]['secular']+preds[0]['transient'])[1] ## CI correction factor for long-term-varying signals
b_smb_lt = sm.tsa.stattools.acf(smb_lowfreq(t_grid))[1]
F_smb_lt = np.sqrt((1+(a_vel_lt*b_smb_lt))/(1-(a_vel_lt*b_smb_lt)))

smb_lt_corr_amax = []
smb_lt_lag_amax = []
smb_lt_significance = []
for xy, pred in zip(xys, preds):
    corr, lags, ci = Xcorr1D_lt(xy, series_func=smb_lowfreq, series_dates=smb_d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=0, normalize=True, pos_only=True)
    ci_mod = F_smb_lt * np.asarray(ci)
    smb_lt_corr_amax.append(corr[abs(corr).argmax()])
    smb_lt_lag_amax.append(lags[abs(corr).argmax()])
    smb_lt_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

In [ ]:
## Low-frequency runoff variability
rnf_evensampled = runoff_func(t_grid_trimmed).squeeze()
rnf_filtered = ndimage.uniform_filter1d(rnf_evensampled, size=window)
rf_lowfreq = interpolate.UnivariateSpline(t_grid_trimmed, rnf_filtered, s=0)

b_runoff_lt = sm.tsa.stattools.acf(rf_lowfreq(t_grid))[1]
F_runoff_lt = np.sqrt((1+(a_vel_lt*b_runoff_lt))/(1-(a_vel_lt*b_runoff_lt)))

rf_lt_corr_amax = []
rf_lt_lag_amax = []
rf_lt_significance = []
for xy, pred in zip(xys, preds):
    corr, lags, ci = Xcorr1D_lt(xy, series_func=rf_lowfreq, series_dates=d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=0, normalize=True, pos_only=True)
    ci_mod = F_runoff_lt * np.asarray(ci)
    rf_lt_corr_amax.append(corr[abs(corr).argmax()])
    rf_lt_lag_amax.append(lags[abs(corr).argmax()])
    rf_lt_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

In [ ]:
## Low-frequency terminus variability
tm_evensampled = termini_func(t_grid_trimmed).squeeze()
tm_filtered = ndimage.uniform_filter1d(tm_evensampled, size=window)
tf_lowfreq = interpolate.UnivariateSpline(t_grid_trimmed, tm_filtered, s=0)

b_terminus_lt = sm.tsa.stattools.acf(tf_lowfreq(t_grid))[1]
F_terminus_lt = np.sqrt((1+(a_vel_lt*b_terminus_lt))/(1-(a_vel_lt*b_terminus_lt)))

term_lt_corr_amax = []
term_lt_lag_amax = []
terminus_lt_significance = []
for xy, pred in zip(xys, preds):
    corr, lags, ci = Xcorr1D_lt(xy, series_func=tf_lowfreq, series_dates=tm_d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=0, normalize=True, pos_only=True)
    ci_mod = F_terminus_lt *np.asarray(ci)
    term_lt_corr_amax.append(corr[abs(corr).argmax()])
    term_lt_lag_amax.append(lags[abs(corr).argmax()])
    terminus_lt_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

Let's generate another six-panel composite showing the maximum cross-correlation and the lag at which it occurs, similar to Figure 2 in the earlier section, but this time for the long-term-varying components as computed above.  The resulting plot is Figure S6 of the manuscript.

In [ ]:
fig, ((ax1, ax2, ax3), (ax4,ax5,ax6)) = plt.subplots(nrows=2,ncols=3, figsize=(12, 8), 
                                                      # constrained_layout=True, 
                                                      sharex=True, sharey=True,
                                                      gridspec_kw={'wspace':0.01})
    
ax1.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc1 = ax1.scatter(np.asarray(xys)[smb_lt_significance,0], np.asarray(xys)[smb_lt_significance,1], 
                  c=np.asarray(smb_lt_corr_amax)[smb_lt_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax1.scatter(np.asarray(xys)[np.invert(smb_lt_significance),0], np.asarray(xys)[np.invert(smb_lt_significance),1], 
                  c=np.asarray(smb_lt_corr_amax)[np.invert(smb_lt_significance)], cmap=div_colors, marker=sig_markers[1],
                  vmin=corrnorm_min, vmax=corrnorm_max)
# ## set up correctly scaled colorbar
# div1 = make_axes_locatable(ax1)
# cax1 = div1.append_axes("right", size="5%", pad=0.1)
# plt.colorbar(sc1, cax=cax1)
# cb1.ax.set_title('AMax. xcorr')
ax1.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
        ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
        xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
        ylabel='Northing [km]', title='Catchment SMB')

ax2.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc2 = ax2.scatter(np.asarray(xys)[rf_lt_significance,0], np.asarray(xys)[rf_lt_significance,1], 
                  c=np.asarray(rf_lt_corr_amax)[rf_lt_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax2.scatter(np.asarray(xys)[np.invert(rf_lt_significance),0], np.asarray(xys)[np.invert(rf_lt_significance),1], 
                  c=np.asarray(rf_lt_corr_amax)[np.invert(rf_lt_significance)], cmap=div_colors, marker=sig_markers[1],
                  vmin=corrnorm_min, vmax=corrnorm_max)
# ## set up correctly scaled colorbar
# div2 = make_axes_locatable(ax2)
# cax2 = div2.append_axes("right", size="5%", pad=0.1)
# fig.colorbar(sc2, cax=cax2)
# cb2.ax.set_title('AMax. xcorr')
ax2.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
        xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
        title='Catchment runoff')

ax3.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc3 = ax3.scatter(np.asarray(xys)[terminus_lt_significance,0], np.asarray(xys)[terminus_lt_significance,1], 
                  c=np.asarray(term_lt_corr_amax)[terminus_lt_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax3.scatter(np.asarray(xys)[np.invert(terminus_lt_significance),0], np.asarray(xys)[np.invert(terminus_lt_significance),1], 
                  c=np.asarray(term_lt_corr_amax)[np.invert(terminus_lt_significance)], cmap=div_colors, marker=sig_markers[1],
                  vmin=corrnorm_min, vmax=corrnorm_max)
## set up correctly scaled colorbar - one for all xcorr plots
div3 = make_axes_locatable(ax3)
cax3 = div3.append_axes("right", size="5%", pad=0.1)
cb3 = fig.colorbar(sc3, cax=cax3, extend='both')
cb3.ax.set_ylabel('AMax. xcorr')
ax3.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
        xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
        title='Terminus position', aspect=1.)

## SECOND ROW
ax4.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc4 = ax4.scatter(np.asarray(xys)[smb_lt_significance,0], np.asarray(xys)[smb_lt_significance,1], 
                  c=np.asarray(smb_lt_lag_amax)[smb_lt_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax4.scatter(np.asarray(xys)[np.invert(smb_lt_significance),0], np.asarray(xys)[np.invert(smb_lt_significance),1], 
                  c=np.asarray(smb_lt_lag_amax)[np.invert(smb_lt_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)
# ## set up correctly scaled colorbar
# div4 = make_axes_locatable(ax4)
# cax4 = div4.append_axes("right", size="5%", pad=0.1)
# plt.colorbar(sc4, cax=cax4)
# cb1.ax.set_title('Lag [d] at peak xcorr')
ax4.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
        xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]', ylabel='Northing [km]')

ax5.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc5 = ax5.scatter(np.asarray(xys)[rf_lt_significance,0], np.asarray(xys)[rf_lt_significance,1], 
                  c=np.asarray(rf_lt_lag_amax)[rf_lt_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax5.scatter(np.asarray(xys)[np.invert(rf_lt_significance),0], np.asarray(xys)[np.invert(rf_lt_significance),1], 
                  c=np.asarray(rf_lt_lag_amax)[np.invert(rf_lt_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)

# ## set up correctly scaled colorbar
# div5 = make_axes_locatable(ax5)
# cax5 = div5.append_axes("right", size="5%", pad=0.1)
# fig.colorbar(sc5, cax=cax5)
# cb2.ax.set_title('Lag [d] at peak xcorr')
ax5.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
        xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]')

ax6.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc6 = ax6.scatter(np.asarray(xys)[terminus_lt_significance,0], np.asarray(xys)[terminus_lt_significance,1], 
                  c=np.asarray(term_lt_lag_amax)[terminus_lt_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax6.scatter(np.asarray(xys)[np.invert(terminus_lt_significance),0], np.asarray(xys)[np.invert(terminus_lt_significance),1], 
                  c=np.asarray(term_lt_lag_amax)[np.invert(terminus_lt_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)

## set up correctly scaled colorbar
div6 = make_axes_locatable(ax6)
cax6 = div6.append_axes("right", size="5%", pad=0.1)
cb6 = fig.colorbar(sc6, cax=cax6, extend='min')
cb6.ax.set_ylabel('Lag [d] at peak xcorr')
ax6.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
        xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]', aspect=1.)
# plt.tight_layout()
plt.show()
# plt.savefig('/Users/lizz/Desktop/20210204-helheim-longterm_xcorr_lag_composite')

## Examine the influence of subglacial topography

In the above plots, we have seen an interesting pattern emerge: there appears to be a clear separation of "upstream" and "downstream" type behaviour.  There is a topographic feature evidence from the maps that could be creating this separation.  Let's now look at an along-flow profile of the subglacial topography to get a better idea of its shape.  We will also compare the seasonal and long-term-varying correlations of each system variable with velocity, and we'll look at the pattern of velocity along flow.

In [ ]:
## Extract bed topo along flowline
xyvals = np.array([(xh[i], yh[i]) for i in range(len(xh))]) ## a dense sampling, more than 9000 points
bed_vals = [float(B_helheim(xh[i], yh[i])) for i in range(len(xh))]
surface_vals = [float(S_helheim(xh[i], yh[i])) for i in range(len(xh))]
xvals = (0.001*np.array(nifl.ArcArray(xyvals)))

## Create arclength array to plot points along flowline
s = 0.001*np.array(nifl.ArcArray(np.array(xys)))+2. # account for the 2km that was removed

In [ ]:
## Compute mean ice speed for each point
mean_speed = [np.mean(pred['full']) for pred in preds]
scaled_speed = [m/np.mean(mean_speed) for m in mean_speed]
scaled_ticks = (np.array([4.0, 6, 8.0])/np.mean(mean_speed)) - np.mean(scaled_speed) # for display

### Set up a stacked plot

In [ ]:
## Introduce vertical offsets and scaling to make a clean stack with only one set of axes
smb_offset=9000 # how many 'meters' above topography baseline to plot this line
runoff_offset=7000 
terminus_offset=5000
velocity_offset=2500 
scaling=1000  # vertical multiplicative factor to display xcorr on same axes as topo

Note that we are going to take a shortcut in the plotting here: We know from the section above that with our usual settings, _none_ of the cross-correlations with long-term-varying SMB or runoff are significantly different from 0.  Thus, instead of testing each array for significance again, we are going to directly plot crosses for "not significant".  If you change settings above, be aware that you will need to account for any changes in significance below.

TODO: add automated handling as in the 6-panel composites above?

In [ ]:
import datetime
from matplotlib.lines import Line2D

In [ ]:
qual_colors = cm.get_cmap('tab20b')

fig, ax = plt.subplots(1, constrained_layout=True)

# ax.axvline(10, color='k', lw=1.0, ls='--')
ax.axvline(14, color='k', lw=0.5, ls='--')
# ax.plot(10*np.ones_like(s), np.linspace(-1300, smb_offset, len(s)), color='k', lw=1.0, ls='--')
ax.plot(s, smb_offset*np.ones_like(s), color='k', lw=0.5, ls=':')
ax.plot(s, runoff_offset*np.ones_like(s), color='k', lw=0.5, ls=':')
ax.plot(s, terminus_offset*np.ones_like(s), color='k', lw=0.5, ls=':')

## SMB stems
markerline, stemlines, baseline = ax.stem(s, smb_offset+ scaling*np.array(smb_lt_corr_amax),
                                          bottom=smb_offset, basefmt='k:',label='SMB long-term',
                                          markerfmt='x')
plt.setp(stemlines, 'color', qual_colors(1))
plt.setp(stemlines, 'linestyle', 'dotted')
plt.setp(markerline, 'color', qual_colors(1))

markerline, stemlines, baseline = ax.stem(s, smb_offset+ scaling*np.array(smb_corr_amax),
                                          bottom=smb_offset, basefmt='k:',label='SMB')
plt.setp(stemlines, 'color', qual_colors(0))
plt.setp(markerline, 'color', qual_colors(0))


## Runoff stems
markerline, stemlines, baseline = ax.stem(s, runoff_offset+ scaling*np.array(rf_lt_corr_amax),
                                          bottom=runoff_offset, basefmt='k:',label='Runoff long-term',
                                          markerfmt='x')
plt.setp(stemlines, 'color', qual_colors(3))
plt.setp(stemlines, 'linestyle', 'dotted')
plt.setp(markerline, 'color', qual_colors(3))

markerline, stemlines, baseline = ax.stem(s, runoff_offset+ scaling*np.array(runoff_corr_amax),
                                          bottom=runoff_offset, basefmt='k:',label='Runoff')
plt.setp(stemlines, 'color', qual_colors(2))
plt.setp(markerline, 'color', qual_colors(2))


## Terminus stems
markerline, stemlines, baseline = ax.stem(s, terminus_offset+ scaling*np.array(term_lt_corr_amax),
                                          bottom=terminus_offset, basefmt='k:',label='Terminus long-term',
                                          markerfmt='d')
plt.setp(stemlines, 'color', qual_colors(5))
plt.setp(stemlines, 'linestyle', 'dotted')
plt.setp(markerline, 'color', qual_colors(5))

markerline, stemlines, baseline = ax.stem(s, terminus_offset+ scaling*np.array(terminus_corr_amax),
                                          bottom=terminus_offset, basefmt='k:',label='Terminus')
plt.setp(stemlines, 'color', qual_colors(4))
plt.setp(markerline, 'color', qual_colors(4))

## add velocity
ax.plot(s, velocity_offset+scaling*np.array(scaled_speed - np.mean(scaled_speed)), color=qual_colors(17), label='Mean speed', lw=2.5)


ax.plot(xvals, bed_vals, color='saddlebrown', lw=2.0)
ax.plot(xvals, surface_vals, color='darkgrey', lw=2.0)
plt.fill_between(xvals, surface_vals, bed_vals, color='darkgrey', alpha=0.5)
plt.fill_between(xvals, bed_vals, y2=-1300, color='saddlebrown', alpha=0.5, hatch='/')
ax.set(xlim=(25, 0), ylim=(-1300, 10000), aspect=0.002, 
        xlabel='Upstream distance [km]',
        yticks=(-1000, 0, 1000, 
                velocity_offset+scaling*scaled_ticks[1],
                terminus_offset,  
                runoff_offset, 
                smb_offset),
        yticklabels=('-1000', '0', '1000 m a.s.l.', 
                     'Surface speed',
                     'Terminus xcorr',
                     'Runoff xcorr',
                     'SMB xcorr'))
minor_locator = ticker.FixedLocator([velocity_offset+scaling*scaled_ticks[0], velocity_offset+scaling*scaled_ticks[2],
                              terminus_offset-0.5*scaling, terminus_offset+0.5*scaling, 
                              runoff_offset-0.5*scaling, runoff_offset+0.5*scaling,
                              smb_offset-0.5*scaling, smb_offset+0.5*scaling])
minor_formatter = ticker.FixedFormatter([4, '8 km/a',
                                         -0.5, 0.5, 
                                        -0.5, 0.5,
                                        -0.5, 0.5])
ax.yaxis.set_minor_locator(minor_locator)
ax.yaxis.set_minor_formatter(minor_formatter)
ax.tick_params(axis='both', which='major', length=7)
ax.tick_params(axis='y', which='minor', length=4)
ax.get_yticklabels()[3].set_color(qual_colors(17)) # match the ticks to the plotted color
ax.get_yticklabels()[4].set_color(qual_colors(4)) 
ax.get_yticklabels()[5].set_color(qual_colors(2))
ax.get_yticklabels()[6].set_color(qual_colors(0))


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig('/Users/Lizz/Desktop/{}-helheim-along_flow_stack.png'.format(datetime.date.today()), bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots()
mtypes = ['o', 'd', 'x']
labels = ['Short term (sig. at 95%)', 'Long term (sig. at 95%)', 'Not significant']
lines = [Line2D([0], [0], color='k', linewidth=0, marker=m) for m in mtypes]
plt.legend(lines, labels, loc='upper right', bbox_to_anchor=(0.5, 0.5))
for sp in ('top', 'bottom', 'left', 'right'):
    ax.spines[sp].set_visible(False)
    ax.tick_params(which='both', length=0)

## Further analyses

In the supplementary material of our manuscript, we present additional figures including autocorrelation functions and correlograms.  Stay tuned for those plots to be added to this walkthrough.  In the meantime, you can find scripts to produce those figures in the Manuscript-figures folder of [this GitHub repository](http://github.com/ehultee/helheim-fiesta).

In [ ]:
## make a stack, this time with lags included, as requested by reviewer 2

## Introduce vertical offsets and scaling to make a clean stack with only one set of axes
smb_offset=15000 # how many 'meters' above topography baseline to plot this line
smb_lag_offset=13000
runoff_offset=11000 
runoff_lag_offset=9000
terminus_offset=7000
terminus_lag_offset=5000
velocity_offset=2500 
scaling=1000  # vertical multiplicative factor to display xcorr on same axes as topo
lag_scaling=1 # multiplicative factor to display lag on same axes

capped_smb_lags = np.where(np.asarray(smb_lag_amax)>365, 365, smb_lag_amax)
capped_runoff_lags = np.where(np.asarray(runoff_lag_amax)>365, 365, runoff_lag_amax)

qual_colors = cm.get_cmap('tab20b')

fig, ax = plt.subplots(1, constrained_layout=True, figsize=(5,10))

# ax.axvline(10, color='k', lw=1.0, ls='--')
ax.axvline(14, color='k', lw=0.5, ls='--')
# ax.plot(10*np.ones_like(s), np.linspace(-1300, smb_offset, len(s)), color='k', lw=1.0, ls='--')
ax.plot(s, smb_offset*np.ones_like(s), color='k', lw=0.5, ls=':')
ax.plot(s, runoff_offset*np.ones_like(s), color='k', lw=0.5, ls=':')
ax.plot(s, terminus_offset*np.ones_like(s), color='k', lw=0.5, ls=':')

## SMB stems
# markerline, stemlines, baseline = ax.stem(s, smb_offset+ scaling*np.array(smb_lt_corr_amax),
#                                           bottom=smb_offset, basefmt='k:',label='SMB long-term',
#                                           markerfmt='x')
# plt.setp(stemlines, 'color', qual_colors(1))
# plt.setp(stemlines, 'linestyle', 'dotted')
# plt.setp(markerline, 'color', qual_colors(1))

markerline, stemlines, baseline = ax.stem(s, smb_offset+ scaling*np.array(smb_corr_amax),
                                          bottom=smb_offset, basefmt='k:',label='SMB')
plt.setp(stemlines, 'color', qual_colors(0))
plt.setp(markerline, 'color', qual_colors(0))

## SMB lags
markerline, stemlines, baseline = ax.stem(s, smb_lag_offset+ lag_scaling*np.array(capped_smb_lags),
                                          bottom=smb_lag_offset, basefmt='k:',label='SMB lags',
                                          markerfmt='1')
plt.setp(stemlines, 'color', qual_colors(0))
plt.setp(markerline, 'color', qual_colors(0))

## Runoff stems
# markerline, stemlines, baseline = ax.stem(s, runoff_offset+ scaling*np.array(rf_lt_corr_amax),
#                                           bottom=runoff_offset, basefmt='k:',label='Runoff long-term',
#                                           markerfmt='x')
# plt.setp(stemlines, 'color', qual_colors(3))
# plt.setp(stemlines, 'linestyle', 'dotted')
# plt.setp(markerline, 'color', qual_colors(3))

markerline, stemlines, baseline = ax.stem(s, runoff_offset+ scaling*np.array(runoff_corr_amax),
                                          bottom=runoff_offset, basefmt='k:',label='Runoff')
plt.setp(stemlines, 'color', qual_colors(2))
plt.setp(markerline, 'color', qual_colors(2))


## Runoff lags
markerline, stemlines, baseline = ax.stem(s, runoff_lag_offset+ lag_scaling*np.array(capped_runoff_lags),
                                          bottom=runoff_lag_offset, basefmt='k:',label='Runoff lags',
                                          markerfmt='1')
plt.setp(stemlines, 'color', qual_colors(2))
plt.setp(markerline, 'color', qual_colors(2))

## Terminus stems
# markerline, stemlines, baseline = ax.stem(s, terminus_offset+ scaling*np.array(term_lt_corr_amax),
#                                           bottom=terminus_offset, basefmt='k:',label='Terminus long-term',
#                                           markerfmt='d')
# plt.setp(stemlines, 'color', qual_colors(5))
# plt.setp(stemlines, 'linestyle', 'dotted')
# plt.setp(markerline, 'color', qual_colors(5))

markerline, stemlines, baseline = ax.stem(s, terminus_offset+ scaling*np.array(terminus_corr_amax),
                                          bottom=terminus_offset, basefmt='k:',label='Terminus')
plt.setp(stemlines, 'color', qual_colors(4))
plt.setp(markerline, 'color', qual_colors(4))

## Runoff lags
markerline, stemlines, baseline = ax.stem(s, terminus_lag_offset+ lag_scaling*np.array(terminus_lag_amax),
                                          bottom=terminus_lag_offset, basefmt='k:',label='Runoff lags',
                                          markerfmt='1')
plt.setp(stemlines, 'color', qual_colors(4))
plt.setp(markerline, 'color', qual_colors(4))

## add velocity
ax.plot(s, velocity_offset+scaling*np.array(scaled_speed - np.mean(scaled_speed)), color=qual_colors(17), label='Mean speed', lw=2.5)


ax.plot(xvals, bed_vals, color='saddlebrown', lw=2.0)
ax.plot(xvals, surface_vals, color='darkgrey', lw=2.0)
plt.fill_between(xvals, surface_vals, bed_vals, color='darkgrey', alpha=0.5)
plt.fill_between(xvals, bed_vals, y2=-1300, color='saddlebrown', alpha=0.5, hatch='/')
ax.set(xlim=(25, 0), ylim=(-1300, 16000), aspect=0.002, 
        xlabel='Upstream distance [km]',
        yticks=(-1000, 0, 1000, 
                velocity_offset+scaling*scaled_ticks[1],
                terminus_lag_offset,
                terminus_offset,
                runoff_lag_offset,
                runoff_offset, 
                smb_lag_offset,
                smb_offset),
        yticklabels=('-1000', '0', '1000 m a.s.l.', 
                     'Surface speed',
                     'Terminus lag',
                     'Terminus xcorr',
                     'Runoff lag',
                     'Runoff xcorr',
                     'SMB lag',
                     'SMB xcorr'))
minor_locator = ticker.FixedLocator([velocity_offset+scaling*scaled_ticks[0], velocity_offset+scaling*scaled_ticks[2],
                                     terminus_lag_offset+365,
                              terminus_offset-0.5*scaling, terminus_offset+0.5*scaling, 
                              runoff_lag_offset+365,
                              runoff_offset-0.5*scaling, runoff_offset+0.5*scaling,
                              smb_lag_offset+365,
                              smb_offset-0.5*scaling, smb_offset+0.5*scaling])
minor_formatter = ticker.FixedFormatter([4, '8 km/a',
                                         '365 d',
                                          -0.5, 0.5, 
                                         '365 d',
                                        -0.5, 0.5,
                                        '365 d',
                                        -0.5, 0.5])
ax.yaxis.set_minor_locator(minor_locator)
ax.yaxis.set_minor_formatter(minor_formatter)
ax.tick_params(axis='both', which='major', length=7)
ax.tick_params(axis='y', which='minor', length=4)
ax.get_yticklabels()[3].set_color(qual_colors(17)) # match the ticks to the plotted color
for j in (4,5):
    ax.get_yticklabels()[j].set_color(qual_colors(4)) 
for j in (6,7):
    ax.get_yticklabels()[j].set_color(qual_colors(2))
for j in (8,9):
    ax.get_yticklabels()[j].set_color(qual_colors(0))


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig('/Users/Lizz/Desktop/{}-helheim-along_flow_lag_stack.png'.format(datetime.date.today()), bbox_inches='tight', dpi=300)